In [1]:
'''
Created by Avik Bhattacharya
CMPS 4720/6720 Homework: Implement your own classification algorithm
'''

import sys
print "Welcome to %s\n\n"%sys.version
print '===Importing the necessary libraries for the project==='
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

Welcome to 2.7.14 |Anaconda custom (64-bit)| (default, Oct  5 2017, 02:28:52) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


===Importing the necessary libraries for the project===


In [2]:
'''
Sign function is used to activate the neuron
'''
def activation(x):
    if x>0:
        return 1
    else:
        return 0

In [3]:
'''
Stochastic gradient descent used for learning weights, online training
'''
def train_algorithm(data,n_iter,learn_rate):
    weights=[]
    '''
    The weight vector is initialised with all zeros
    '''
    for i in range(1,np.shape(data)[1]):
        weights.append(0)
    '''
    n_iter is the terminating condition
    '''
    for i in range (0,n_iter):
        '''
        k iterates over all the examples one by one for each value of i
        Data has been read and manipulated to pad x0 with 1 and also shift the label to the last column.
        (Please see the function: read_source)
        '''
        for k in range(0,len(data)):
            inputs=data[k][0:np.shape(data)[1]-1]
            label=data[k][np.shape(data)[1]-1:np.shape(data)[1]]
            adder=0
            for i in range (1,np.shape(data)[1]):
                adder += inputs[i-1]*weights[i-1]
            '''
            Perceptron delta learning rule implemented
            '''
            guess=activation((adder))
            
            error=(label-guess)
            #print "Guess=%d ,Target=%d, Error=%d"%(guess,label ,error)
            '''
            Update the weight on the basis of error generated in recongnizing the k-th examples 
            '''
            weights=weights+learn_rate*(error*inputs)
    return weights   

In [4]:
'''
Learned weights used for sample classification using trained model
'''
def test_algorithm(data,w):
    predicted=[]
    weights=w
    '''
    Take all k examples one by one and predict the labels using the trained neuron
    '''
    for k in range(0,len(data)):
        inputs=data[k][0:np.shape(data)[1]-1]
        adder=0
        for i in range(1,np.shape(data)[1]):
            adder += inputs[i-1]*weights[i-1]
        guess=activation(adder)
        predicted.append(guess)
    '''
    return the predicted labels for the k examples to the main function
    '''
    return predicted

In [5]:
'''
Accuracy, TP, TN, FP, FN are evaluated
'''
def metrics(predicted, labels):
    tp=0.0
    fp=0.0
    tn=0.0
    fn=0.0
    '''
    Checking the predictions and comparing them to the actual labels one by one
    '''
    for i in range(0,len(labels)):
        if (labels[i]==1):
            if (predicted[i]==1):
                tp+=1
            else:
                fn+=1
        if (labels[i]==0):
            if (predicted[i]==0):
                tn+=1
            else:
                fp+=1
    
    accuracy=(tp+tn)*1.0/(tp+tn+fp+fn)
    
    return accuracy,tp,tn,fp,fn

In [6]:
'''
Function to return a feature set and labels ready to train classifier 
'''
def read_source(filename):
    df1=pd.read_csv(filename, header=None)
    data=df1.values
    '''
    for loop used below swaps first and last column because initially
    the model was developed considering the last column to be label,
    so modified it to keep up with the pattern earlier developed
    '''
    for i in range (0,len(data)):
        temp=data[i][0]
        data[i][0]=data[i][np.shape(data)[1]-1]
        data[i][np.shape(data)[1]-1]=temp
    s=(np.shape(data)[0],np.shape(data)[1]+1)
    a=np.ones(s)
    '''
    
    '''
    '''
    for i in range(0,len(data)):
        if (data[i][np.shape(data)[1]-1]==0):
            data[i][np.shape(data)[1]-1]=0
    '''
    '''
    padding extra 1s at x0 positions
    '''
    for i in range (0,np.shape(data)[1]):
        for j in range(0,len(data)):
            a[j][i+1]=data[j][i]
    data=a
    '''
    Return totally processed feature set and labels in the sense that the x0 is 
    padded with 1 and the last column is made the labels.
    '''
    return data

In [7]:
'''
def validate_set(arr,percentage):
    np.random.shuffle(arr)
    if((percentage*1.0/100)*(len(arr))==0):
        r=1
    else:
        r=int(((percentage*1.0/100)*(len(arr))))
    m = len(arr)-r
    k = np.shape(arr)[1]
    s=(m,k)
    test=np.zeros(s)
    size=(r,np.shape(arr)[1])
    val=np.zeros(size)
    for i in range(0,r):
        val[i][0:np.shape(arr)[1]]=arr[i][0:np.shape(arr)[1]]
    for i in range(r,(len(arr))):
        test[i-r][0:np.shape(arr)[1]]=arr[i][0:np.shape(arr)[1]]
    
    return test,val
'''

'\ndef validate_set(arr,percentage):\n    np.random.shuffle(arr)\n    if((percentage*1.0/100)*(len(arr))==0):\n        r=1\n    else:\n        r=int(((percentage*1.0/100)*(len(arr))))\n    m = len(arr)-r\n    k = np.shape(arr)[1]\n    s=(m,k)\n    test=np.zeros(s)\n    size=(r,np.shape(arr)[1])\n    val=np.zeros(size)\n    for i in range(0,r):\n        val[i][0:np.shape(arr)[1]]=arr[i][0:np.shape(arr)[1]]\n    for i in range(r,(len(arr))):\n        test[i-r][0:np.shape(arr)[1]]=arr[i][0:np.shape(arr)[1]]\n    \n    return test,val\n'

In [8]:
'''
Plotter function
'''
'''
def plotter(start,end,step):
    final=[]
    
    y_axis_err=[]
    percentage=20
    print '\n Randomly select %d percent of training set for tuning the number of iterations required\n'%(percentage)
    print 'Validating on 10 randomly selected models'
    #train_data=read_source('train.txt')
    #train_data,val_data=validate_set(train_data,percentage)
    learn_rate=0.001
    for k in range(0,10):
        y_axis=[]
        train_data=read_source('train.txt')
        train_data,val_data=validate_set(train_data,percentage)
        for n_iter in range(start,end,step):
            w=train_algorithm(train_data,n_iter,learn_rate)
            test_data=val_data
            predicted=test_algorithm(test_data,w)
            labels=[]
            for i in range(0,len(test_data)):
                labels.append(test_data[i][np.shape(test_data)[1]-1])
                labels[i]=int(labels[i])
            accuracy,tp,tn,fp,fn= metrics(predicted,labels)
            err=(1-accuracy)
            y_axis.append(accuracy)
            y_axis_err.append(err)
        xaxis = np.arange(start,end,step)
        fig, ax = plt.subplots()
        ax.plot(xaxis, y_axis, 'b')
        ax.set_xlabel('No. of iterations (n_iter)')
        ax.set_ylabel('Classification Accuracy')
        plt.title('Accuracy w.r.t n_iter')
        plt.savefig('n_iter vs acc.pdf')
        plt.show()
        ymax=max(y_axis)
        for i in range(start,end,step):
            if(y_axis[i]==ymax and  y_axis[i-1]!=y_axis[i]):
                n=i
        print 'Best accuracy of %d th model is at n_ter=%d'%(k+1,n)
        final.append(n)
    return final
 '''

"\ndef plotter(start,end,step):\n    final=[]\n    \n    y_axis_err=[]\n    percentage=20\n    print '\n Randomly select %d percent of training set for tuning the number of iterations required\n'%(percentage)\n    print 'Validating on 10 randomly selected models'\n    #train_data=read_source('train.txt')\n    #train_data,val_data=validate_set(train_data,percentage)\n    learn_rate=0.001\n    for k in range(0,10):\n        y_axis=[]\n        train_data=read_source('train.txt')\n        train_data,val_data=validate_set(train_data,percentage)\n        for n_iter in range(start,end,step):\n            w=train_algorithm(train_data,n_iter,learn_rate)\n            test_data=val_data\n            predicted=test_algorithm(test_data,w)\n            labels=[]\n            for i in range(0,len(test_data)):\n                labels.append(test_data[i][np.shape(test_data)[1]-1])\n                labels[i]=int(labels[i])\n            accuracy,tp,tn,fp,fn= metrics(predicted,labels)\n            err=(1-

# Main Code Here

In [9]:



'''
copy pasted the training and testing file into text files and saved them
Name of Training set file: train.txt
Name of Testing set file: test.txt
'''
train_data=read_source('train.txt')
test_data=read_source('test.txt')
n_iter=56 #on the basis of changing values and checking accuracy, didn't do CV
learn_rate=0.1 # for AND function change this to 1 to make it converge quicker
w=train_algorithm(train_data,n_iter,learn_rate)
print "The weight vector of length %d for the trained perceptron is:\n"%(len(w))
print w
print "\n"
predicted=test_algorithm(test_data,w)
labels=[]
'''
Extracting the labels of the test set and storing them in the array 'labels'
'''
for i in range(0,len(test_data)):
    labels.append(test_data[i][np.shape(test_data)[1]-1])
    labels[i]=int(labels[i])
'''
Calling the metrics function to get the performance metrices
'''
accuracy,tp,tn,fp,fn= metrics(predicted,labels)
print "The model built on the basis of %d iterations performs as below\n"%(n_iter)
print "\n1.Accuracy=%f%% \n2.tp=%d \n3.tn=%d \n4.fp=%d \n5.fn=%d\n"%(accuracy*100,tp,tn,fp,fn)
print "This is our confusion matrix\n"
print "\tFalse\tTrue\nFalse\t%d\t%d\nTrue\t%d\t%d\n"%(tn,fp,fn,tp)
print "Done\n"

The weight vector of length 23 for the trained perceptron is:

[ -7.00000000e-01   1.00000000e-01  -1.00000000e-01  -7.00000000e-01
  -5.00000000e-01  -2.00000000e-01  -2.00000000e-01   5.00000000e-01
   8.00000000e-01   1.00000000e+00  -2.00000000e-01  -1.00000000e-01
   1.00000000e-01  -4.00000000e-01   3.00000000e-01   0.00000000e+00
  -2.00000000e-01  -1.00000000e-01   5.00000000e-01   1.00000000e+00
   1.00000000e-01   3.00000000e-01  -2.77555756e-17]


The model built on the basis of 56 iterations performs as below


1.Accuracy=46.524064% 
2.tp=72 
3.tn=15 
4.fp=0 
5.fn=100

This is our confusion matrix

	False	True
False	15	0
True	100	72

Done

